In [ ]:
!pip install python-dotenv
!pip install openai huggingface_hub transformers
!pip install --upgrade pip
!pip install --upgrade transformers

In [69]:
import sys
import os
from dotenv import load_dotenv
import time
from tqdm import tqdm
import re
import json

opj = os.path.join

In [70]:
import huggingface_hub
import transformers
import torch
from openai import OpenAI

print("Torch Version:", torch.__version__)
print("CUDA is available:", torch.cuda.is_available())

Torch Version: 2.2.0a0+81ea7a4
CUDA is available: True


In [71]:
load_dotenv(verbose=True)

True

In [ ]:
print(os.getenv("OPENAI_API_KEY"))
print(os.getenv("HUGGINGFACE_TOKEN"))
print(os.getenv("HF_HOME"))

In [73]:
client = OpenAI()

In [74]:
prompt = """Your task is to optimize SQL queries for the chinook database. Use the provided query rewriting rules to rewrite queries for optimal performance without changing their intent or results. You have access to a detailed table schema for the chinook database. The input SQL query is LLM's translation of natural language into SQL queries, which can be somewhat inefficient. Ensure each optimized query is efficient, leveraging indexing and avoiding unnecessary operations. For queries that can't be optimized, output as is. Output only one the most optimized query in ```sql``` statement.

## Table Schema of chinook Benchmark
```sql
CREATE TABLE Album
(
    AlbumId INT NOT NULL,
    Title VARCHAR(160) NOT NULL,
    ArtistId INT NOT NULL,
    CONSTRAINT PK_Album PRIMARY KEY  (AlbumId)
);

CREATE TABLE Artist
(
    ArtistId INT NOT NULL,
    name VARCHAR(120),
    CONSTRAINT PK_Artist PRIMARY KEY  (ArtistId)
);

CREATE TABLE Customer
(
    CustomerId INT NOT NULL,
    FirstName VARCHAR(40) NOT NULL,
    LastName VARCHAR(20) NOT NULL,
    Company VARCHAR(80),
    Address VARCHAR(70),
    City VARCHAR(40),
    State VARCHAR(40),
    Country VARCHAR(40),
    PostalCode VARCHAR(10),
    Phone VARCHAR(24),
    Fax VARCHAR(24),
    Email VARCHAR(60) NOT NULL,
    SupportRepId INT,
    CONSTRAINT PK_Customer PRIMARY KEY  (CustomerId)
);

CREATE TABLE Employee
(
    EmployeeId INT NOT NULL,
    LastName VARCHAR(20) NOT NULL,
    FirstName VARCHAR(20) NOT NULL,
    Title VARCHAR(30),
    ReportsTo INT,
    BirthDate TIMESTAMP,
    HireDate TIMESTAMP,
    Address VARCHAR(70),
    City VARCHAR(40),
    State VARCHAR(40),
    Country VARCHAR(40),
    PostalCode VARCHAR(10),
    Phone VARCHAR(24),
    Fax VARCHAR(24),
    Email VARCHAR(60),
    CONSTRAINT PK_Employee PRIMARY KEY  (EmployeeId)
);

CREATE TABLE Genre
(
    GenreId INT NOT NULL,
    Name VARCHAR(120),
    CONSTRAINT PK_Genre PRIMARY KEY  (GenreId)
);

CREATE TABLE invoice
(
    InvoiceId INT NOT NULL,
    CustomerId INT NOT NULL,
    InvoiceDate TIMESTAMP NOT NULL,
    BillingAddress VARCHAR(70),
    BillingCity VARCHAR(40),
    BillingState VARCHAR(40),
    BillingCountry VARCHAR(40),
    BillingPostalCode VARCHAR(10),
    Total NUMERIC(10,2) NOT NULL,
    CONSTRAINT PK_Invoice PRIMARY KEY  (InvoiceId)
);

CREATE TABLE InvoiceLine
(
    InvoiceLineId INT NOT NULL,
    InvoiceId INT NOT NULL,
    TrackId INT NOT NULL,
    UnitPrice NUMERIC(10,2) NOT NULL,
    Quantity INT NOT NULL,
    CONSTRAINT PK_InvoiceLine PRIMARY KEY  (InvoiceLineId)
);

CREATE TABLE MediaType
(
    MediaTypeId INT NOT NULL,
    Name VARCHAR(120),
    CONSTRAINT PK_MediaType PRIMARY KEY  (MediaTypeId)
);

CREATE TABLE Playlist
(
    PlaylistId INT NOT NULL,
    Name VARCHAR(120),
    CONSTRAINT PK_Playlist PRIMARY KEY  (PlaylistId)
);

CREATE TABLE PlaylistTrack
(
    PlaylistId INT NOT NULL,
    TrackId INT NOT NULL,
    CONSTRAINT PK_PlaylistTrack PRIMARY KEY  (PlaylistId, TrackId)
);

CREATE TABLE Track
(
    TrackId INT NOT NULL,
    Name VARCHAR(200) NOT NULL,
    AlbumId INT,
    MediaTypeId INT NOT NULL,
    GenreId INT,
    Composer VARCHAR(220),
    Milliseconds INT NOT NULL,
    Bytes INT,
    UnitPrice NUMERIC(10,2) NOT NULL,
    CONSTRAINT PK_Track PRIMARY KEY  (TrackId)
);
```

## Index and Foreign Key Cascade
```sql
ALTER TABLE Album ADD CONSTRAINT album_ArtistId_fkey
    FOREIGN KEY (ArtistId) REFERENCES Artist (ArtistId) ON DELETE NO ACTION ON UPDATE NO ACTION;

CREATE INDEX IFK_AlbumArtistId ON album (ArtistId);

ALTER TABLE Customer ADD CONSTRAINT customer_SupportRepId_fkey
    FOREIGN KEY (SupportRepId) REFERENCES Employee (EmployeeId) ON DELETE NO ACTION ON UPDATE NO ACTION;

CREATE INDEX IFK_CustomerSupportRepId ON customer (SupportRepId);

ALTER TABLE Employee ADD CONSTRAINT employee_ReportsTo_fkey
    FOREIGN KEY (ReportsTo) REFERENCES Employee (EmployeeId) ON DELETE NO ACTION ON UPDATE NO ACTION;

CREATE INDEX IFK_EmployeeReportsTo ON Employee (ReportsTo);

ALTER TABLE invoice ADD CONSTRAINT invoice_CustomerId_fkey
    FOREIGN KEY (CustomerId) REFERENCES Customer (CustomerId) ON DELETE NO ACTION ON UPDATE NO ACTION;

CREATE INDEX IFK_InvoiceCustomerId ON Invoice (CustomerId);

ALTER TABLE InvoiceLine ADD CONSTRAINT InvoiceLine_InvoiceId_fkey
    FOREIGN KEY (InvoiceId) REFERENCES Invoice (InvoiceId) ON DELETE NO ACTION ON UPDATE NO ACTION;

CREATE INDEX IFK_InvoiceLineInvoiceId ON InvoiceLine (InvoiceId);

ALTER TABLE InvoiceLine ADD CONSTRAINT InvoiceLine_TrackId_fkey
    FOREIGN KEY (TrackId) REFERENCES Track (TrackId) ON DELETE NO ACTION ON UPDATE NO ACTION;

CREATE INDEX IFK_InvoiceLineTrackId ON InvoiceLine (TrackId);

ALTER TABLE PlaylistTrack ADD CONSTRAINT PlaylistTrack_PlaylistId_fkey
    FOREIGN KEY (PlaylistId) REFERENCES Playlist (PlaylistId) ON DELETE NO ACTION ON UPDATE NO ACTION;

CREATE INDEX IFK_PlaylistTrackPlaylistId ON PlaylistTrack (PlaylistId);

ALTER TABLE PlaylistTrack ADD CONSTRAINT PlaylistTrack_TrackId_fkey
    FOREIGN KEY (TrackId) REFERENCES Track (TrackId) ON DELETE NO ACTION ON UPDATE NO ACTION;

CREATE INDEX IFK_PlaylistTrackTrackId ON PlaylistTrack (TrackId);

ALTER TABLE Track ADD CONSTRAINT track_AlbumId_fkey
    FOREIGN KEY (AlbumId) REFERENCES Album (AlbumId) ON DELETE NO ACTION ON UPDATE NO ACTION;

CREATE INDEX IFK_TrackAlbumId ON Track (AlbumId);

ALTER TABLE Track ADD CONSTRAINT track_GenreId_fkey
    FOREIGN KEY (GenreId) REFERENCES Genre (GenreId) ON DELETE NO ACTION ON UPDATE NO ACTION;

CREATE INDEX IFK_TrackGenreId ON Track (GenreId);

ALTER TABLE Track ADD CONSTRAINT track_MediaTypeId_fkey
    FOREIGN KEY (MediaTypeId) REFERENCES MediaType (MediaTypeId) ON DELETE NO ACTION ON UPDATE NO ACTION;

CREATE INDEX IFK_TrackMediaTypeId ON Track (MediaTypeId);
```

## Input chinook SQL Query
```sql
{{INPUT_SQL_QUERY}}
```

Optimized Rewritten SQL Query: """

In [75]:
def gpt_call(model_id, system=None, prompt="", verbose=False):
    start_time = time.time()

    if system is None:
        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    else:
        messages = [
            {
                "role": "system", 
                "content": [
                    {
                        "type": "text", 
                        "text": system
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]

    assert model_id in ["gpt-4o", "gpt-4o-mini", "gpt-4-turbo"], "wrong model id provided."
    assert len(prompt.strip()) > 0, "prompt must not be empty string."

    if verbose:
        print("Model:", model_id)
        print("Input Message:", messages)

    response = client.chat.completions.create(
        model=model_id,
        messages=messages,
        temperature=1,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        response_format={
            "type": "text"
        }
    )

    if verbose:
        print("Model Response:", response.choices[0].message.content)

    print(f"Time Elapsed: {time.time() - start_time:.5f} sec")

    return response.choices[0].message.content

In [76]:
# test gpt
gpt_call(model_id="gpt-4o", system=None, prompt="Hello", verbose=True)

Model: gpt-4o
Input Message: [{'role': 'user', 'content': [{'type': 'text', 'text': 'Hello'}]}]
Model Response: Hello! How can I assist you today?
Time Elapsed: 0.48871 sec


'Hello! How can I assist you today?'

In [77]:
def read_sql_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {file_path}")
    except Exception as e:
        print(f"파일을 읽는 중 오류가 발생했습니다: {e}")

def write_sql_file(content, file_path):
    try:
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(content)
            print(f"SQL 파일이 저장되었습니다: {file_path}")
    except Exception as e:
        print(f"파일을 저장하는 중 오류가 발생했습니다: {e}")

In [78]:
original_query_dict = dict()

In [79]:
ORIGINAL_QUERY_DIR = "/home/work/hoeun/spider1/chinook_queries/"
OUTPUT_DIR = "/home/work/hoeun/spider1/gpt4_rewrite_queries/"
QUERY_RANGE = range(0, 84)

In [80]:
for qnum in QUERY_RANGE:
    file_path = opj(ORIGINAL_QUERY_DIR, f"{qnum}.sql")
    sql_content = read_sql_file(file_path)

    if sql_content is not None:
        original_query_dict[qnum] = sql_content

In [81]:
for key, content in original_query_dict.items():
    print(f"{key}m.sql\n==========\n{content}\n==========\n\n")

0m.sql
-- db_id: chinook_1
-- question: How many albums are there?
SELECT count(*) FROM ALBUM



1m.sql
-- db_id: chinook_1
-- question: Find the number of albums.
SELECT count(*) FROM ALBUM



2m.sql
-- db_id: chinook_1
-- question: List the names of all music genres.
SELECT Name FROM GENRE



3m.sql
-- db_id: chinook_1
-- question: What are the names of different music genres?
SELECT Name FROM GENRE



4m.sql
-- db_id: chinook_1
-- question: Find all the customer information in state NY.
SELECT * FROM CUSTOMER WHERE State  =  'NY'



5m.sql
-- db_id: chinook_1
-- question: What is all the customer information for customers in NY state?
SELECT * FROM CUSTOMER WHERE State  =  'NY'



6m.sql
-- db_id: chinook_1
-- question: What are the first names and last names of the employees who live in Calgary city.
SELECT FirstName ,  LastName FROM EMPLOYEE WHERE City  =  'Calgary'



7m.sql
-- db_id: chinook_1
-- question: Find the full names of employees living in the city of Calgary.
SELECT Fi

In [82]:
def rewrite_query():
    rewritten_query_list = []

    for qnum in tqdm(QUERY_RANGE):
        query = original_query_dict[qnum]

        query_prompt = prompt.replace("{{INPUT_SQL_QUERY}}", query)

        rewrite_start_time = time.time()
        response = gpt_call(model_id="gpt-4-turbo", system=None, prompt=query_prompt, verbose=False)
        rewrite_time = time.time() - rewrite_start_time

        rewritten_query_list.append({"query": response, "time": rewrite_time})

    return rewritten_query_list

In [83]:
rewritten_query = rewrite_query()

  1%|          | 1/84 [00:00<01:18,  1.06it/s]

Time Elapsed: 0.94481 sec


  2%|▏         | 2/84 [00:02<01:24,  1.03s/it]

Time Elapsed: 1.08095 sec


  4%|▎         | 3/84 [00:02<01:14,  1.09it/s]

...

Time Elapsed: 2.78334 sec


 99%|█████████▉| 83/84 [03:35<00:03,  3.11s/it]

Time Elapsed: 5.36914 sec


100%|██████████| 84/84 [03:42<00:00,  2.64s/it]

Time Elapsed: 6.56365 sec


In [86]:
print(rewritten_query)

[{'query': '```sql\nSELECT count(*) FROM Album;\n```', 'time': 0.9448933601379395}, {'query': '```sql\nSELECT count(*) FROM Album;\n```', 'time': 1.0810391902923584}, {'query': '```sql\nSELECT Name FROM Genre;\n```', 'time': 0.7855658531188965}, {'query': '```sql\nSELECT Name FROM Genre\n```', 'time': 0.8057162761688232}, {'query': "```sql\nSELECT CustomerId, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId \nFROM Customer \nWHERE State = 'NY';\n```", 'time': 1.9828109741210938}, {'query': "```sql\nSELECT CustomerId, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId \nFROM Customer \nWHERE State = 'NY'\n```", 'time': 5.312546968460083}, {'query': "```sql\n-- The original query is already very efficient as it has targeted only necessary columns and uses a direct filter on the WHERE clause.\nSELECT FirstName, LastName FROM Employee WHERE City = 'Calgary';\n```", 'time': 2.087798

In [87]:
times = [str(x['time']) for x in rewritten_query]
print(times)

['0.9448933601379395', '1.0810391902923584', '0.7855658531188965', '0.8057162761688232', '1.9828109741210938', '5.312546968460083', '2.0877981185913086', '3.0138165950775146', '1.160654067993164', '0.9204404354095459', '1.7673060894012451', '2.397291898727417', '2.021507740020752', '1.8902297019958496', '1.7414500713348389', '1.9556527137756348', '1.8531734943389893', '1.6168813705444336', '3.49963641166687', '3.525038719177246', '2.1889007091522217', '4.901218414306641', '0.7873749732971191', '1.3572328090667725', '1.916416883468628', '1.3852901458740234', '4.976027965545654', '1.7798502445220947', '2.666826009750366', '2.7064032554626465', '4.830986261367798', '2.918207883834839', '6.106687068939209', '5.428703546524048', '1.2251226902008057', '1.5060148239135742', '3.184544324874878', '2.2855770587921143', '2.0526986122131348', '5.136552572250366', '1.0757207870483398', '3.345667839050293', '2.3309061527252197', '2.213239908218384', '2.2686285972595215', '2.4898688793182373', '5.052

In [88]:
with open("./times.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(times))

In [89]:
def extract_sql_blocks(text):
    # ```sql ... ``` 사이의 텍스트를 추출하는 정규표현식 패턴
    pattern = r"```sql\s(.*?)```"
    # 패턴과 일치하는 모든 텍스트를 리스트로 반환
    sql_blocks = re.findall(pattern, text, re.DOTALL)
    return sql_blocks

In [90]:
for i in tqdm(QUERY_RANGE):
    query = rewritten_query[i]['query']
    
    extracted_query = extract_sql_blocks(query)

    with open(f"./gpt4_rewrite_queries/{i}.sql", "w", encoding="utf-8") as f:
        f.write(extracted_query[0])

100%|██████████| 84/84 [00:00<00:00, 1426.03it/s]
